In [ ]:
import pandas as pd
import numpy as np
import pickle
import sys
from sentence_transformers import SentenceTransformer
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import Sequential, regularizers 

from keras.layers import Embedding, LSTM, Dense
from keras import losses
from keras.layers import Dense
from keras.layers import Flatten, InputLayer

import time
from keras.callbacks import EarlyStopping, ModelCheckpoint
import torch
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn import metrics
import plotly.express as px
from transformers import AutoTokenizer, AutoModel


In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.4 MB/s 
     |████████████████████████████████| 5.8 MB 32.1 MB/s 
     |████████████████████████████████| 1.3 MB 22.4 MB/s 
     |████████████████████████████████| 182 kB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 13.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9c99a06f6d104d22ca052ee56337fc302108db43835409fc4c36f280ab50761a
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_df = "/content/drive/MyDrive/fake_bert/politifact_global_feature.pkl"

In [ ]:
df = pickle.load(open(path_df, "rb"))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
fea_imp_df = df[['L5', 'L6', 'S6', 'Sen1', 'Sen14', 'Sen18', 'T11', 'T4', 'T5', 'T9']]

In [ ]:
x_train, x_test, y_train, y_test, f_train, f_test = train_test_split(df['comp_text'], df['target'], fea_imp_df, test_size= .20, random_state= 111, shuffle=False )

In [ ]:
def cls_pooling(model_output, attention_mask):
    return model_output[0][:,0]

def get_embeds(embedd):
  embed_tok = tokenizer(embedd.tolist(), padding=True, truncation=True, max_length=512, return_tensors='pt')

  with torch.no_grad():
    model_output = model(**embed_tok)

#Perform pooling. In this case, mean pooling
  # sentence_embeddings = mean_pooling(model_output, embed_tok['attention_mask'])
  # mean_pooled = model_output.sum(axis=1) / attention_mask.sum(axis=-1).unsqueeze(-1)
  sentence_embeddings = cls_pooling(model_output, embed_tok['attention_mask'])
  # tf.convert_to_tensor(sentence_embeddings, dtype='float32')
  return tf.convert_to_tensor(sentence_embeddings, dtype='float32')

In [ ]:
embed_train = get_embeds(x_train)
embed_test = get_embeds(x_test)

In [ ]:
import pickle
pickle.dump((embed_train, embed_test), open("/content/drive/MyDrive/fake_bert/sbert-embed.pkl", "wb"))


In [ ]:
scaler = preprocessing.MinMaxScaler()
f_train_transform = scaler.fit_transform(f_train)
f_test_transform = scaler.fit_transform(f_test)

In [ ]:
merged_train_arr = np.hstack([embed_train, f_train_transform])

In [ ]:
merged_test_arr = np.hstack([embed_test, f_test_transform])

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
cv=5
gs_xgb = XGBClassifier()


param_grid = [
    {'booster':['gbtree', 'dart'], 'n_estimators':[100, 150, 200, 250], 'max_depth':[5, 6, 7]}
]
grid = GridSearchCV(gs_xgb, param_grid, refit=True, cv=cv, verbose=3, scoring='roc_auc')
            

In [ ]:
grid.fit(merged_train_arr, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.837 total time=   1.4s
[CV 2/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.891 total time=   0.6s
[CV 3/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.830 total time=   0.6s
[CV 4/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.891 total time=   0.7s
[CV 5/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.943 total time=   0.6s
[CV 1/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.845 total time=   0.8s
[CV 2/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.891 total time=   0.8s
[CV 3/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.837 total time=   0.8s
[CV 4/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.888 total time=   0.8s
[CV 5/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.945 total time=   0.8s
[CV 1/5] END boost

GridSearchCV(cv=5, estimator=XGBClassifier(),
             param_grid=[{'booster': ['gbtree', 'dart'], 'max_depth': [5, 6, 7],
                          'n_estimators': [100, 150, 200, 250]}],
             scoring='roc_auc', verbose=3)

In [ ]:
predicted_output = grid.predict(merged_test_arr)

In [ ]:
pickle.dump(grid, open("/content/drive/MyDrive/fake_bert/grid_feaimp.pkl", "wb"))

In [ ]:
print(grid.best_params_)

{'booster': 'gbtree', 'max_depth': 7, 'n_estimators': 250}


In [ ]:
print('Accuracy:', accuracy_score(predicted_output, y_test))
print('F1 score:', f1_score(y_test, predicted_output))
print('Recall:', recall_score(y_test, predicted_output))
print('Precision:', precision_score(y_test, predicted_output))
print('ROC_AUC Score:', roc_auc_score(y_test, predicted_output))
print(classification_report(y_test, predicted_output))

Accuracy: 0.8571428571428571
F1 score: 0.8571428571428571
Recall: 0.84375
Precision: 0.8709677419354839
ROC_AUC Score: 0.857358870967742
              precision    recall  f1-score   support

           0       0.84      0.87      0.86        31
           1       0.87      0.84      0.86        32

    accuracy                           0.86        63
   macro avg       0.86      0.86      0.86        63
weighted avg       0.86      0.86      0.86        63

